In [ ]:
import pandas as pd
import numpy as np
import os
import time
import cv2
import random
import keras
import matplotlib.pyplot as plt
from PIL import Image,ImageOps
from skimage.transform import resize
from skimage.io import imread, imshow
from skimage.feature import hog
from skimage import exposure
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Convolution2D , MaxPooling2D, Flatten, Dense , Dropout
from keras.layers import Activation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.layers import BatchNormalization
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from keras import callbacks
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from sklearn.metrics import accuracy_score

In [ ]:
def make_dataframe():
    path = 'rawframes'
    row = 64
    col = 64
    column_names = ['pixel'+str(i) for i in range(row*col)]
    column_names_with_label = column_names.copy()
    column_names_with_label.append('label')
    numclass=0
    numfol=0
    files=0
    filec=0
    twod = []
    count = 0

    for class_name in os.listdir(path):
        classcount=0
        if(classcount==51):break
        if(class_name!='.DS_Store'):
            numfol=0
            for fol in os.listdir(path+'/'+class_name):
                if(fol[-9:]!='.DS_Store'):
                    numfol+=1
                    if(numfol==30):break
                    numfil=0
                    for filename in os.listdir(path+'/'+class_name+'/'+fol):
                        numfil+=1
                        if(filename[-9:] != '.DS_Store' and numfil%10==0):
                            
                            img = Image.open(path+'/'+class_name+'/'+fol+'/'+filename).convert('L')
                            img = img.resize((col,row))
                            img = np.array(img).flatten()
                                
                            dict_ = {column_names[i]: img[i] for i in range(len(column_names))}
                            dict_['label'] = class_name
                            twod.append(list(dict_.values()))
                            count+=1
                            if(count == 30000): break
                        
                            print(count)          
           
    df = pd.DataFrame(twod,columns=column_names_with_label)
    df.to_csv('data.csv',index=None)


In [ ]:
def hog_data():
    data = pd.read_csv('data.csv')
    hogarr=[]
    
    for index,row in data.iterrows():
        img = np.reshape(np.array(row[:-1],dtype=int),(64,64)).astype('uint8')
        hog_feature, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),block_norm= 'L2', visualize=True, multichannel=False)
        hogarr.append(list(hog_feature))    
    hogdf = pd.DataFrame(hogarr, columns=['pixel'+str(i) for i in range(len(hog_feature))])
    
    extra_column = data['label']
    hogdf = pd.concat([hogdf,extra_column], axis = 1)
   
    hogdf.to_csv('HOGdata.csv',index=None)
  
    hog_y = pd.DataFrame(hogdf , columns =['label'])
   
    hog_le = LabelEncoder()
    hog_y = hog_le.fit_transform(hog_y)
    
    hog_train =hogdf
    hog_train = hog_train.drop('label', axis=1).values
     
    
    #----------------------------------------------------------
    
    print('hog extraction completed .... Working on classification')
    
    X_train, X_test, y_train, y_test = train_test_split(hog_train, hog_y, test_size=0.2, shuffle = True)
    
    
        # *************************** SVM + HOG *********************************
    normalized_x_train_svm = pd.DataFrame(scaler.fit_transform(X_train))
    normalized_x_test_svm = pd.DataFrame(scaler.fit_transform(X_test))
   
    svm = LinearSVC()
    svm.fit(normalized_x_train_svm,y_train)
    svm_preds = svm.predict(normalized_x_test_svm)
    svm_acc_hog = accuracy_score(y_test,svm_preds)
    
    
    # ************************************ HOG + DNN ********************************
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    XX_train = X_train[:,0:].reshape(X_train.shape[0],X_train[1].size).astype('float32')
        
    XX_train = XX_train / 255.0
    
    
    XX_test = X_test[:,0:].reshape(X_test.shape[0],X_test[1].size).astype('float32')
    XX_test = XX_test / 255.0
    
    
    classifier = Sequential()
  
    classifier.add(Dense(output_dim = 512, input_shape=(1764,)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.4))    


    classifier.add(Dense(output_dim = 256))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.4))    
    

    classifier.add(Dense(output_dim = 128))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.4))    
    
    # classifier.add(Flatten())
    classifier.add(Dense(output_dim = 51, activation= 'softmax'))
    
    classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    
    classifier.summary()
    
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
   
    
   
    history = classifier.fit(XX_train,y_train,
                    validation_data= (XX_test,y_test),
                    batch_size=32, epochs = 100 ,
                    verbose=1,
                    callbacks=[lr_reducer, early_stopper ])
    
    classifier.save_weights('hog_and_cnn.h5')
    
    acc_hog = history.history['acc']
    val_acc_hog = history.history['val_acc']
    loss_hog = history.history['loss']
    val_loss_hog = history.history['val_loss']
    
    epochs_hog = range(len(acc_hog))    

    return(epochs_hog, acc_hog, loss_hog, val_acc_hog, val_loss_hog,svm_acc_hog)

In [ ]:
def SIFT_data():
    data = pd.read_csv('data.csv')
    siftarr=[]
    mindescriptors=1000
    eps=1e-7

    for index,row in data.iterrows():
        img = np.reshape(np.array(row[:-1],dtype=int),(64,64)).astype('uint8')
        sift = cv2.xfeatures2d.SURF_create(10)
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if(descriptors is not None):
            mindescriptors = min(mindescriptors,descriptors.shape[0])

    for index,row in data.iterrows():
        img = np.reshape(np.array(row[:-1],dtype=int),(64,64)).astype('uint8')
        sift = cv2.xfeatures2d.SIFT_create(nfeatures= mindescriptors,
                            nOctaveLayers=3,
                            contrastThreshold=0.04,
                            edgeThreshold=10.0,
                            sigma=1.6) #nfeatures=mindescriptors)
        keypoints, descriptors = sift.detectAndCompute(img, None)
        
        if(descriptors is not None):
            descriptors=descriptors[:mindescriptors]
            sift_feature = np.array(descriptors).flatten()
            siftarr.append(list(sift_feature))

    siftdf = pd.DataFrame(siftarr,columns=['pixel'+str(i) for i in range(len(sift_feature))])
    
    extra_column = data['label']
    siftdf = pd.concat([siftdf,extra_column], axis = 1)
    siftdf.fillna(siftdf.median(), inplace=True)
    
    siftdf.to_csv('SIFTdata.csv',index=None)
    
    sift_y = pd.DataFrame(siftdf , columns =['label'])
   
    sift_le = LabelEncoder()
    sift_y = sift_le.fit_transform(sift_y)
    
    sift_train =siftdf
    sift_train = sift_train.drop('label', axis=1).values
        
    
     # --------------------------
    print('SIFT extraction completed .... Working on CNN')
    
    X_train, X_test, y_train, y_test = train_test_split(sift_train, sift_y, test_size=0.2, shuffle = True)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    XX_train = X_train[:,0:].reshape(X_train.shape[0],128).astype('float32')
        
    XX_train = XX_train / 255.0
    
    
    XX_test = X_test[:,0:].reshape(X_test.shape[0],128).astype('float32')
    XX_test = XX_test / 255.0
    
    
    classifier = Sequential()
    
    #  classifier.add(Flatten())
    classifier.add(Dense(output_dim = 512, input_shape=(128,)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2))    

    classifier.add(Dense(output_dim = 256))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2))    


    classifier.add(Dense(output_dim = 128))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2))    
    
    # classifier.add(Flatten())
    classifier.add(Dense(output_dim = 51, activation= 'softmax'))
    
    classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    
    classifier.summary()
    
    
    #filename='my_model_sift_cnn_data.csv'
    #filepath="my_model_SIFT_CNN_Best-weights-my_model-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"

    #csv_log=callbacks.CSVLogger(filename, separator=',', append=False)
    #checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    #callbacks_list = [csv_log,checkpoint]
    #callbacks_list = [csv_log]
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
   
    
    
    history = classifier.fit(XX_train,y_train,
                    validation_data= (XX_test,y_test),
                    batch_size=32, epochs = 100 ,
                    verbose=1,
                    callbacks=[lr_reducer, early_stopper])
   
    classifier.save_weights('sift_and_cnn.h5')

    acc_sift = history.history['acc']
    val_acc_sift = history.history['val_acc']
    loss_sift = history.history['loss']
    val_loss_sift = history.history['val_loss']
    
    epochs_sift = range(len(acc_sift))

    return (epochs_sift,acc_sift,loss_sift, val_acc_sift,val_loss_sift)

In [ ]:
def cnn_data():
    # Importing the dataset
    dataset = pd.read_csv('data.csv')
    data_encoded = dataset
    row = len(dataset)
    col = len(dataset.columns)

    number_classes=  data_encoded['label'].value_counts()
    

    le = LabelEncoder()
    data_encoded['label'] = le.fit_transform(data_encoded['label'])
    X = dataset.iloc[:, 0:col-1].values
    y = dataset.iloc[:, col-1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, shuffle=True)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    XX_train = X_train[:,0:].reshape(X_train.shape[0],64,64,1).astype('float32')
        
    XX_train = XX_train / 255.0
    
    
    XX_test = X_test[:,0:].reshape(X_test.shape[0],64,64,1).astype('float32')
    XX_test = XX_test / 255.0
    
    
    classifier = Sequential()
    classifier.add(Convolution2D(64,(3,3),input_shape = (64,64,1)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Convolution2D(64,(3,3),input_shape = (64,64,1)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Dropout(0.4))    
   
    classifier.add(Convolution2D(128,(3,3)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Convolution2D(128,(3,3)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Dropout(0.4))    
    
    classifier.add(Convolution2D(256,(3,3)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Convolution2D(128,(3,3)))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Dropout(0.4))    
    
    
    
    classifier.add(Flatten())
    classifier.add(Dense(output_dim = 256))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.4))    

    classifier.add(Dense(output_dim = 128))
    classifier.add(Activation('relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.4))    
    
    
    classifier.add(Dense(output_dim = number_classes.shape[0] , activation= 'softmax'))
    
    classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    
    classifier.summary()
    
 
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
    
    history= classifier.fit(XX_train,y_train,
                   validation_data = (XX_test,y_test),
                   batch_size=32,
                   epochs = 100 , 
                   verbose=1,
                   callbacks=[lr_reducer, early_stopper])

    classifier.save_weights('only_cnn.h5')
    
    acc_cnn = history.history['acc']
    val_acc_cnn = history.history['val_acc']
    loss_cnn = history.history['loss']
    val_loss_cnn = history.history['val_loss']
    
    epochs_cnn = range(len(acc_cnn))
    
    return(epochs_cnn,acc_cnn, loss_cnn, val_acc_cnn, val_loss_cnn)

In [ ]:
print('*****************************************************')
print('making dataframe')
make_dataframe()




In [ ]:
print('*****************************************************')
print('SIFT keypoints and descriptors extraction')
epochs_sift,acc_sift,loss_sift, val_acc_sift,val_loss_sift = SIFT_data()


In [ ]:
print('*****************************************************')
print('HOG feature extraction')
epochs_hog, acc_hog, loss_hog, val_acc_hog, val_loss_ho, svm_acc_hog = hog_data()    


In [ ]:
print('*****************************************************')
print('CNN')
epochs_cnn,acc_cnn, loss_cnn, val_acc_cnn, val_loss_cnn = cnn_data()

In [ ]:
# -------------------------- SIFT + CNN ------------------------------------

#print('training accuracy of SIFT-CNN: {:.2f} '.format(acc_sift*100) + '% ')
#print('testing accuracy of SIFT-CNN:{:.2f} '.format(val_acc_sift*100) + '% ')

plt.plot(epochs_sift,acc_sift , 'r' , label = "Training accuracy")
plt.plot(epochs_sift,val_acc_sift , 'b' , label = "Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
    
plt.plot(epochs_sift,loss_sift , 'r' , label = "Training loss")
plt.plot(epochs_sift,val_loss_sift , 'b' , label = "Validation loss")
plt.title('Training and Validation loss')
plt.legend()
plt.figure()
plt.show()

In [ ]:
# ---------------------------- HOG + CNN ---------------------

# ************** change the index at the last epoch the training ended in print statement .

print('training accuracy of HOG-CNN:')
#print((acc_hog[epochs_hog[23]]))
print('validation accuracy of HOG-CNN:')
#print((val_acc_hog[epochs_hog[23]]))

print("accuracy of SVM HOG".format(svm_acc_hog))

plt.plot(epochs_hog,acc_hog , 'r' , label = "Training accuracy")
plt.plot(epochs_hog,val_acc_hog , 'b' , label = "Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs_hog,loss_hog , 'r' , label = "Training loss")
plt.plot(epochs_hog,val_loss_hog , 'b' , label = "Validation loss")
plt.title('Training and Validation loss')
plt.legend()
plt.figure()
plt.show()


In [ ]:
# -------------------------- CNN -----------------------

print('Training accuracy of CNN:{:.2f}'.format(acc_cnn*100) + '% ')
print('Testing accuracy of CNN:{:.2f}'.format(val_acc_cnn*100) + '% ')
        

plt.plot(epochs_cnn,acc_cnn , 'r' , label = "Training accuracy")
plt.plot(epochs_cnn,val_acc_cnn , 'b' , label = "Validation accuracy")
plt.title('Training and Validation accuracy')
    
plt.plot(epochs_cnn,loss_cnn , 'r' , label = "Training loss")
plt.plot(epochs_cnn,val_loss_cnn , 'b' , label = "Validation loss")
plt.title('Training and Validation loss')
plt.legend()
plt.figure()
plt.show()